In [1]:
import numpy as np
import faiss
import torch
from tqdm.notebook import tqdm

In [7]:
! pip3 install torch

You should consider upgrading via the '/usr/bin/python3.6 -m pip install --upgrade pip' command.


In [2]:



class FaissKNeighbors:
    def __init__(self, k=5):
        self.index = None
        self.y = None
        self.k = k

    def fit(self, X, y):
        self.index = faiss.IndexFlatL2(X.shape[1])
        self.index.add(X.astype(np.float32))
        res = faiss.StandardGpuResources()
#         self.gpu_index = faiss.index_cpu_to_all_gpus(  # build the index
#             self.index
#         )
        self.gpu_index = faiss.index_cpu_to_gpu(  # build the index
            res,
            1,
            self.index
        )
        self.y = y

    def predict(self, X):
        distances, indices = self.gpu_index.search(X.astype(np.float32), k=self.k)
        votes = self.y[indices]
        predictions = np.array([np.argmax(np.bincount(x)) for x in votes])
        return indices

In [3]:
knn = FaissKNeighbors(k=50)

a = np.random.normal(size=(500000, 500))
b = np.zeros(500000, dtype=np.int64)
b[250000:] = 1
# knn = KNeighborsClassifier(n_neighbors=50)
knn.fit(a, b)

In [22]:
for i in tqdm(range(500000)):
    c = knn.predict(a[i:i+1])

In [14]:
c

(array([0]),
 array([[  0.     , 731.0241 , 731.3761 , 736.7205 , 750.9963 , 751.1914 ,
         754.35767, 761.94836, 765.2665 , 765.72327, 766.0238 , 766.69775,
         766.91504, 768.5211 , 770.8925 , 771.50055, 773.3078 , 773.68445,
         774.2686 , 775.9426 , 775.9465 , 776.175  , 776.4354 , 776.5114 ,
         776.6543 , 776.78314, 777.80554, 778.4669 , 778.61554, 778.67596,
         778.6988 , 778.9269 , 779.0544 , 779.6638 , 780.6416 , 781.067  ,
         781.1879 , 781.3655 , 781.46454, 782.21326, 782.6823 , 782.7848 ,
         782.84467, 782.89526, 782.95905, 783.5349 , 783.635  , 783.7126 ,
         783.8795 , 783.8924 ]], dtype=float32),
 array([[   170,  77946, 277146, 184262, 127610, 364259,  40157,  99971,
         381385, 395540, 352188, 421782,   4195, 296721, 167775, 180594,
         321346,  84428,  77747, 143536,  52592, 429831, 322516, 270430,
          60863, 339763, 150254,  94708, 417267,  51336, 305694, 299147,
          90819,  39054, 128439, 311597, 19914